# Modelo e Submissão para Kaggle
## Jigsaw Toxic Comment Classification Challenge

Este notebook treina um modelo e gera o arquivo de submissão para o Kaggle.

## 1. Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier

# Processamento de texto
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

print("Bibliotecas importadas com sucesso!")

# Baixar recursos do NLTK (se necessário)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)

## 2. Carregamento dos Dados

In [ ]:
# Definir caminhos
data_dir = Path('../data/raw')

# Carregar dados
print("Carregando dados de treino...")
train_df = pd.read_csv(data_dir / 'train.csv')
print(f"Shape: {train_df.shape}")

print("\nCarregando dados de teste...")
test_df = pd.read_csv(data_dir / 'test.csv')
print(f"Shape: {test_df.shape}")

print("\nCarregando sample submission...")
sample_submission = pd.read_csv(data_dir / 'sample_submission.csv')
print(f"Shape: {sample_submission.shape}")

# Colunas de toxicidade
toxic_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print(f"\nClasses de toxicidade: {toxic_cols}")

## 3. Pré-processamento de Texto

In [ ]:
def preprocess_text(text):
    """
    Pré-processa o texto:
    - Remove URLs
    - Remove caracteres especiais
    - Converte para minúsculas
    - Remove espaços extras
    """
    if pd.isna(text):
        return ""
    
    # Converter para string
    text = str(text)
    
    # Remover URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remover caracteres especiais, manter apenas letras, números e espaços
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Converter para minúsculas
    text = text.lower()
    
    # Remover espaços extras
    text = ' '.join(text.split())
    
    return text

# Aplicar pré-processamento
print("Pré-processando textos de treino...")
train_df['comment_text_processed'] = train_df['comment_text'].apply(preprocess_text)

print("Pré-processando textos de teste...")
test_df['comment_text_processed'] = test_df['comment_text'].apply(preprocess_text)

print("\nExemplo de texto original:")
print(train_df['comment_text'].iloc[0][:200])
print("\nExemplo de texto processado:")
print(train_df['comment_text_processed'].iloc[0][:200])

## 4. Feature Engineering - TF-IDF

In [ ]:
# Criar vetorizador TF-IDF
# Limitar o número de features para não consumir muita memória
max_features = 5000

print(f"Criando vetorizador TF-IDF com max_features={max_features}...")
vectorizer = TfidfVectorizer(
    max_features=max_features,
    ngram_range=(1, 2),  # Unigramas e bigramas
    min_df=2,  # Palavra deve aparecer em pelo menos 2 documentos
    max_df=0.95,  # Palavra não deve aparecer em mais de 95% dos documentos
    stop_words='english'
)

# Treinar o vetorizador e transformar dados de treino
print("\nTreinando vetorizador e transformando dados de treino...")
X_train = vectorizer.fit_transform(train_df['comment_text_processed'])
print(f"Shape dos dados de treino: {X_train.shape}")

# Transformar dados de teste
print("\nTransformando dados de teste...")
X_test = vectorizer.transform(test_df['comment_text_processed'])
print(f"Shape dos dados de teste: {X_test.shape}")

# Labels
y_train = train_df[toxic_cols].values
print(f"\nShape dos labels: {y_train.shape}")

## 5. Divisão dos Dados (Treino/Validação)

In [ ]:
# Dividir dados de treino em treino e validação
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, 
    test_size=0.2, 
    random_state=42,
    stratify=None  # Não estratificar porque temos múltiplas classes
)

print(f"Dados de treino: {X_train_split.shape}")
print(f"Dados de validação: {X_val_split.shape}")

## 6. Treinamento do Modelo

In [ ]:
# Usar OneVsRestClassifier com LogisticRegression
# Isso treina um classificador binário para cada classe
print("Treinando modelo (OneVsRest + LogisticRegression)...")
print("Isso pode levar alguns minutos...")

model = OneVsRestClassifier(
    LogisticRegression(
        max_iter=1000,
        random_state=42,
        solver='lbfgs',
        C=1.0
    ),
    n_jobs=-1  # Usar todos os cores disponíveis
)

# Treinar o modelo
model.fit(X_train_split, y_train_split)

print("\nModelo treinado com sucesso!")

## 7. Avaliação do Modelo

In [ ]:
# Fazer previsões no conjunto de validação
print("Fazendo previsões no conjunto de validação...")
y_val_pred = model.predict_proba(X_val_split)

# Calcular ROC-AUC para cada classe
print("\n=== Métricas de Validação ===")
scores = {}
for i, col in enumerate(toxic_cols):
    score = roc_auc_score(y_val_split[:, i], y_val_pred[:, i])
    scores[col] = score
    print(f"{col:20s}: {score:.4f}")

# Média dos scores
mean_score = np.mean(list(scores.values()))
print(f"\n{'Média':20s}: {mean_score:.4f}")
print("\n(Quanto maior, melhor. Score máximo é 1.0)")

## 8. Previsões no Conjunto de Teste

In [ ]:
# Treinar modelo final com todos os dados de treino
print("Treinando modelo final com todos os dados de treino...")
model_final = OneVsRestClassifier(
    LogisticRegression(
        max_iter=1000,
        random_state=42,
        solver='lbfgs',
        C=1.0
    ),
    n_jobs=-1
)

model_final.fit(X_train, y_train)
print("Modelo final treinado!")

# Fazer previsões no conjunto de teste
print("\nFazendo previsões no conjunto de teste...")
test_predictions = model_final.predict_proba(X_test)
print(f"Shape das previsões: {test_predictions.shape}")

## 9. Criar Arquivo de Submissão

In [ ]:
# Criar DataFrame de submissão
submission = pd.DataFrame({
    'id': test_df['id']
})

# Adicionar previsões para cada classe
for i, col in enumerate(toxic_cols):
    submission[col] = test_predictions[:, i]

# Verificar formato
print("=== Formato da Submissão ===")
print(f"Shape: {submission.shape}")
print(f"\nPrimeiras linhas:")
display(submission.head())

print(f"\nEstatísticas das previsões:")
display(submission[toxic_cols].describe())

# Verificar se está no formato correto
print(f"\nColunas esperadas: {list(sample_submission.columns)}")
print(f"Colunas criadas: {list(submission.columns)}")
assert list(submission.columns) == list(sample_submission.columns), "Colunas não correspondem!"
print("\n✓ Formato correto!")

## 10. Salvar Arquivo de Submissão

In [ ]:
# Criar diretório para submissões
submission_dir = Path('../submissions')
submission_dir.mkdir(exist_ok=True)

# Salvar arquivo de submissão
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'submission_{timestamp}.csv'
filepath = submission_dir / filename

submission.to_csv(filepath, index=False)
print(f"Arquivo de submissão salvo em: {filepath}")
print(f"Tamanho do arquivo: {filepath.stat().st_size / 1024:.2f} KB")

# Também salvar como submission.csv (mais fácil de encontrar)
submission.to_csv(submission_dir / 'submission.csv', index=False)
print(f"\nTambém salvo como: {submission_dir / 'submission.csv'}")

## 11. Instruções para Submeter no Kaggle

### Passo a Passo para Submeter no Kaggle:

1. **Acesse a página da competição:**
   - https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

2. **Vá para a aba "Submit Predictions"** (ou "Submeter Previsões")

3. **Clique em "Upload Submission File"**

4. **Selecione o arquivo:**
   - `submissions/submission.csv` ou o arquivo com timestamp

5. **Clique em "Make Submission"**

6. **Aguarde o processamento** (pode levar alguns minutos)

7. **Veja seu score!** O Kaggle mostrará sua pontuação (ROC-AUC médio)

### Dicas:
- Você pode submeter até 5 vezes por dia
- O score público é calculado em uma parte do conjunto de teste
- O score privado (final) é calculado após o encerramento da competição
- Tente melhorar o modelo e submeter novamente!

## 12. Próximos Passos para Melhorar o Modelo

### Ideias para melhorar a pontuação:

1. **Pré-processamento mais avançado:**
   - Lemmatização/Stemming
   - Remoção de stopwords mais inteligente
   - Tratamento de emojis e caracteres especiais

2. **Feature Engineering:**
   - Aumentar max_features do TF-IDF
   - Usar n-gramas maiores (trigramas)
   - Adicionar features de comprimento do texto
   - Word embeddings (Word2Vec, GloVe, FastText)

3. **Modelos mais avançados:**
   - XGBoost, LightGBM, CatBoost
   - Redes Neurais (LSTM, BERT, DistilBERT)
   - Ensemble de modelos

4. **Otimização de hiperparâmetros:**
   - GridSearch ou RandomizedSearch
   - Validação cruzada

5. **Tratamento de classes desbalanceadas:**
   - SMOTE
   - Class weights
   - Focal Loss